## Example submission

Image Matching Challenge 2025: https://www.kaggle.com/competitions/image-matching-challenge-2025

This notebook creates a simple submission using ALIKED and LightGlue, plus DINO for shortlisting, on GPU. Adapted from [last year](https://www.kaggle.com/code/oldufo/imc-2024-submission-example).

Remember to select an accelerator on the sidebar to the right, and to disable internet access when submitting a notebook to the competition.

In [ ]:
# IMPORTANT 
#Install dependencies and copy model weights to run the notebook without internet access when submitting to the competition.

!pip install --no-index /kaggle/input/imc2024-packages-lightglue-rerun-kornia/* --no-deps
!mkdir -p /root/.cache/torch/hub/checkpoints
!cp /kaggle/input/aliked/pytorch/aliked-n16/1/aliked-n16.pth /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/lightglue/pytorch/aliked/1/aliked_lightglue.pth /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/lightglue/pytorch/aliked/1/aliked_lightglue.pth /root/.cache/torch/hub/checkpoints/aliked_lightglue_v0-1_arxiv-pth

In [ ]:
import sys
import os
from tqdm import tqdm
from time import time, sleep
import gc
import numpy as np
import h5py
import dataclasses
import pandas as pd
from IPython.display import clear_output
from collections import defaultdict
from copy import deepcopy
from PIL import Image

import cv2
import torch
import torch.nn.functional as F
import kornia as K
import kornia.feature as KF

import torch
from lightglue import match_pair
from lightglue import ALIKED, LightGlue
from lightglue.utils import load_image, rbd
from transformers import AutoImageProcessor, AutoModel

# IMPORTANT Utilities: importing data into colmap and competition metric
import pycolmap
sys.path.append('/kaggle/input/imc25-utils')
from database import *
from h5_to_db import *
import metric

In [ ]:

print("PyTorch version:", torch.__version__)
import sys
print("Python version:", sys.version)

print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("Device count:", torch.cuda.device_count())
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))


In [ ]:
# Do not forget to select an accelerator on the sidebar to the right.
device = K.utils.get_cuda_device_if_available(0)
print(f'{device=}')

In [ ]:
def load_torch_image(fname, device=torch.device('cpu')):
    img = K.io.load_image(fname, K.io.ImageLoadType.RGB32, device=device)[None, ...]
    return img


# Must Use efficientnet global descriptor to get matching shortlists.
def get_global_desc(fnames, device = torch.device('cpu')):
    processor = AutoImageProcessor.from_pretrained('/kaggle/input/dinov2/pytorch/large/1')
    model = AutoModel.from_pretrained('/kaggle/input/dinov2/pytorch/large/1')
    model = model.eval()
    model = model.to(device)
    global_descs_dinov2 = []
    for i, img_fname_full in tqdm(enumerate(fnames),total= len(fnames)):
        key = os.path.splitext(os.path.basename(img_fname_full))[0]
        timg = load_torch_image(img_fname_full)
        with torch.inference_mode():
            inputs = processor(images=timg, return_tensors="pt", do_rescale=False).to(device)
            outputs = model(**inputs)
            dino_mac = F.normalize(outputs.last_hidden_state[:,1:].max(dim=1)[0], dim=1, p=2)
        global_descs_dinov2.append(dino_mac.detach().cpu())
    global_descs_dinov2 = torch.cat(global_descs_dinov2, dim=0)
    return global_descs_dinov2


def get_img_pairs_exhaustive(img_fnames):
    index_pairs = []
    for i in range(len(img_fnames)):
        for j in range(i+1, len(img_fnames)):
            index_pairs.append((i,j))
    return index_pairs


def get_image_pairs_shortlist(fnames,
                              sim_th = 0.6, # should be strict
                              min_pairs = 30,
                              exhaustive_if_less = 20,
                              device=torch.device('cpu')):
    num_imgs = len(fnames)
    if num_imgs <= exhaustive_if_less:
        return get_img_pairs_exhaustive(fnames)
    descs = get_global_desc(fnames, device=device)
    dm = torch.cdist(descs, descs, p=2).detach().cpu().numpy()

    # 只分析上三角（去掉对角线），避免重复
    triu_indices = np.triu_indices_from(dm, k=1)
    dm_flat = dm[triu_indices]
    
    # 打印统计信息
    print("Distance Matrix Statistics:")
    print(f"Min:  {dm_flat.min():.4f}")
    print(f"Max:  {dm_flat.max():.4f}")
    print(f"Mean: {dm_flat.mean():.4f}")
    print(f"Std:  {dm_flat.std():.4f}")
    print(f"20%:  {np.percentile(dm_flat, 20):.4f}")
    print(f"25%:  {np.percentile(dm_flat, 25):.4f}")
    print(f"50%:  {np.percentile(dm_flat, 50):.4f}")
    print(f"USED 75%:  {np.percentile(dm_flat, 75):.4f}")
    threshold = dm_flat.mean() + np.sqrt(3) * dm_flat.std()
    # removing half
    thr = min(np.percentile(dm_flat, 75), sim_th)
    mask = dm <= thr
    print("thr :", thr)
    # mask = dm<=threshold
    total = 0
    matching_list = []
    ar = np.arange(num_imgs)
    already_there_set = []
    for st_idx in range(num_imgs-1):
        mask_idx = mask[st_idx]
        to_match = ar[mask_idx]
        if len(to_match) < min_pairs:
            to_match = np.argsort(dm[st_idx])[:min_pairs]  
        for idx in to_match:
            if st_idx == idx:
                continue
            if dm[st_idx, idx] < threshold:
                matching_list.append(tuple(sorted((st_idx, idx.item()))))
                total+=1
    matching_list = sorted(list(set(matching_list)))
    return matching_list

def detect_aliked(img_fnames,
                  feature_dir = '.featureout',
                  num_features = 4096,
                  resize_to = 1024,
                  device=torch.device('cpu')):
    dtype = torch.float32 # ALIKED has issues with float16
    extractor = ALIKED(max_num_keypoints=num_features, detection_threshold=0.005).eval().to(device, dtype)
    extractor.preprocess_conf["resize"] = resize_to
    if not os.path.isdir(feature_dir):
        os.makedirs(feature_dir)
    with h5py.File(f'{feature_dir}/keypoints.h5', mode='w') as f_kp, \
         h5py.File(f'{feature_dir}/descriptors.h5', mode='w') as f_desc:
        for img_path in tqdm(img_fnames):
            img_fname = img_path.split('/')[-1]
            key = img_fname
            with torch.inference_mode():
                image0 = load_torch_image(img_path, device=device).to(dtype)
                feats0 = extractor.extract(image0)  # auto-resize the image, disable with resize=None
                kpts = feats0['keypoints'].reshape(-1, 2).detach().cpu().numpy()
                descs = feats0['descriptors'].reshape(len(kpts), -1).detach().cpu().numpy()
                f_kp[key] = kpts
                f_desc[key] = descs
    return

def match_with_lightglue(img_fnames,
                   index_pairs,
                   feature_dir = '.featureout',
                   device=torch.device('cpu'),
                   min_matches=20,verbose=True):
    lg_matcher = KF.LightGlueMatcher("aliked", {"width_confidence": -1,
                                                "depth_confidence": -1,
                                                 "mp": True if 'cuda' in str(device) else False}).eval().to(device)
    with h5py.File(f'{feature_dir}/keypoints.h5', mode='r') as f_kp, \
        h5py.File(f'{feature_dir}/descriptors.h5', mode='r') as f_desc, \
        h5py.File(f'{feature_dir}/matches.h5', mode='w') as f_match:
        for pair_idx in tqdm(index_pairs):
            idx1, idx2 = pair_idx
            fname1, fname2 = img_fnames[idx1], img_fnames[idx2]
            key1, key2 = fname1.split('/')[-1], fname2.split('/')[-1]
            kp1 = torch.from_numpy(f_kp[key1][...]).to(device)
            kp2 = torch.from_numpy(f_kp[key2][...]).to(device)
            desc1 = torch.from_numpy(f_desc[key1][...]).to(device)
            desc2 = torch.from_numpy(f_desc[key2][...]).to(device)
            with torch.inference_mode():
                dists, idxs = lg_matcher(desc1,
                                         desc2,
                                         KF.laf_from_center_scale_ori(kp1[None]),
                                         KF.laf_from_center_scale_ori(kp2[None]))
            if len(idxs)  == 0:
                continue
            n_matches = len(idxs)
            if verbose:
                print (f'{key1}-{key2}: {n_matches} matches')
            group  = f_match.require_group(key1)
            if n_matches >= min_matches:
                 group.create_dataset(key2, data=idxs.detach().cpu().numpy().reshape(-1, 2))
    return



def import_into_colmap(img_dir, feature_dir ='.featureout', database_path = 'colmap.db'):
    db = COLMAPDatabase.connect(database_path)
    db.create_tables()
    single_camera = False
    fname_to_id = add_keypoints(db, feature_dir, img_dir, '', 'simple-pinhole', single_camera)
    add_matches(
        db,
        feature_dir,
        fname_to_id,
    )
    db.commit()
    return

In [ ]:
def load_image_names_from_json(cluster_path):
    with open(os.path.join(cluster_path, 'images.json'), 'r') as f:
        full_paths = json.load(f)  # 可能是 ['/path/to/images/img001.jpg', ...]
        image_names = [os.path.basename(p) for p in full_paths]  # 提取 'img001.jpg'
    return image_names

In [ ]:
def import_into_colmap_cluster(
    img_dir,
    cluster_path='.featureout/cluster_0',
    database_path = '.featureout/cluster_0/colmap.db',
    image_names = None
):
    """
    Import keypoints and matches into COLMAP database using helper functions.

    Args:
        img_dir (str): Directory containing image files
        cluster_path (str): Path with matches.h5
        database_path (str): Output database location
        image_names (list[str]): Optional subset of image names to include
    """
    db = COLMAPDatabase.connect(database_path)
    db.create_tables()
    single_camera = False
    # Add keypoints and images
    fname_to_id = add_keypoints(
        db=db,
        h5_path=cluster_path,
        image_path=img_dir,
        img_ext='',
        camera_model='simple-pinhole',
        single_camera=single_camera
    )
    # Filter fname_to_id to only use the selected subset (if provided)
    if image_names is not None:
        fname_to_id = {k: v for k, v in fname_to_id.items() if k in image_names}

    # Add matches between selected image pairs
    add_matches(
        db=db,
        h5_path=cluster_path,
        fname_to_id=fname_to_id
    )
    db.commit()
    db.close()

In [ ]:
import json


def run_colmap_clusters(
    feature_dir,
    images_dir,
    timings
):
    """
    Run COLMAP reconstruction for each cluster folder in feature_dir.

    Parameters:
    - feature_dir: Directory containing cluster_* folders with images.json, keypoints.h5, matches.h5
    - images_dir: Path to raw image files
    - timings: dict to record durations
    - min_model_size: minimum model size for COLMAP mapping
    - max_num_models: maximum number of maps to attempt
    """
    cluster_folders = sorted([f for f in os.listdir(feature_dir) if f.startswith('cluster_')])
    all_maps = []
    for i, folder in enumerate(cluster_folders):
        cluster_path = os.path.join(feature_dir, folder)
        database_path = os.path.join(cluster_path, 'colmap.db')
        image_list_path = os.path.join(cluster_path, 'images.json')

        if not os.path.isfile(image_list_path):
            print(f"[Cluster {i}] Missing images.json, skipping.")
            continue

        with open(image_list_path, 'r') as f:
            image_names = [os.path.basename(x) for x in json.load(f)]

        if os.path.isfile(database_path):
            os.remove(database_path)

        gc.collect()
        sleep(1)

        # Step 1: import keypoints, matches into COLMAP db
        import_into_colmap_cluster(
            img_dir=images_dir,
            cluster_path=cluster_path,
            database_path=database_path,
            image_names=image_names
        )

        # Step 2: RANSAC
        t = time()
        pycolmap.match_exhaustive(database_path)
        t_ransac = time() - t
        timings['RANSAC'].append(t_ransac)
        print(f'[Cluster {i}] Ran RANSAC in {t_ransac:.4f} sec')

        # Step 3: Incremental mapping
        output_path = os.path.join(cluster_path, 'colmap_rec_aliked')
        os.makedirs(output_path, exist_ok=True)

        mapper_options = pycolmap.IncrementalPipelineOptions()
        mapper_options.min_model_size = 6
        mapper_options.max_num_models = 4
        mapper_options.mapper.filter_max_reproj_error	 = 2.0
        # mapper_options.init_max_error = 6.0

        t = time()
        maps = pycolmap.incremental_mapping(
            database_path=database_path,
            image_path=images_dir,
            output_path=output_path,
            options=mapper_options
        )
        t_rec = time() - t
        timings['Reconstruction'].append(t_rec)
        print(f'[Cluster {i}] Reconstruction done in {t_rec:.4f} sec')
        all_maps.append(maps)
    
    return timings, all_maps


In [ ]:
# import networkx as nx

# import networkx as nx
# from tqdm import tqdm

# def match_with_lightglue_and_cluster(
#     img_fnames,
#     index_pairs,
#     feature_dir='.featureout',
#     device=torch.device('cpu'),
#     min_matches=20,
#     aliked_dis_min=100,
#     verbose=True
# ):
#     # 初始化 LightGlue 匹配器
#     lg_matcher = KF.LightGlueMatcher(
#         "aliked", {
#             "width_confidence": -1,
#             "depth_confidence": -1,
#             "mp": 'cuda' in str(device)
#         }
#     ).eval().to(device)

#     num_imgs = len(img_fnames)
#     match_graph = nx.Graph()
#     match_graph.add_nodes_from(range(num_imgs))

#     # 加载关键点和描述子，准备写入匹配结果
#     with h5py.File(os.path.join(feature_dir, 'keypoints.h5'), 'r') as f_kp, \
#          h5py.File(os.path.join(feature_dir, 'descriptors.h5'), 'r') as f_desc, \
#          h5py.File(os.path.join(feature_dir, 'matches.h5'), 'w') as f_match:

#         # Step 1: 匹配每对图像
#         for idx1, idx2 in tqdm(index_pairs, desc="LightGlue Matching"):
#             img1 = os.path.basename(img_fnames[idx1])
#             img2 = os.path.basename(img_fnames[idx2])

#             kp1 = torch.from_numpy(f_kp[img1][...]).to(device)
#             kp2 = torch.from_numpy(f_kp[img2][...]).to(device)
#             desc1 = torch.from_numpy(f_desc[img1][...]).to(device)
#             desc2 = torch.from_numpy(f_desc[img2][...]).to(device)

#             with torch.inference_mode():
#                 _, idxs = lg_matcher(
#                     desc1, desc2,
#                     KF.laf_from_center_scale_ori(kp1[None]),
#                     KF.laf_from_center_scale_ori(kp2[None])
#                 )

#             if len(idxs) == 0:
#                 continue

#             n_matches = len(idxs)
#             if verbose:
#                 print(f'{img1} - {img2}: {n_matches} matches')

#             if n_matches >= min_matches:
#                 match_graph.add_edge(idx1, idx2, weight=n_matches)
#                 group = f_match.require_group(img1)
#                 group.create_dataset(img2, data=idxs.detach().cpu().numpy().reshape(-1, 2))

#         # Step 2: 提取聚类（connected components）
#         raw_clusters = list(nx.connected_components(match_graph))
#         final_clusters = []
#         outliers = set()

#         for cluster in raw_clusters:
#             cluster = list(cluster)
#             subgraph = match_graph.subgraph(cluster)
#             valid_nodes = []

#             for node in cluster:
#                 strong_neighbors = [
#                     neighbor for neighbor in subgraph.neighbors(node)
#                     if subgraph[node][neighbor]['weight'] >= aliked_dis_min
#                 ]
#                 if strong_neighbors:
#                     valid_nodes.append(node)
#                 else:
#                     outliers.add(node)

#             if len(valid_nodes) >= 2:
#                 final_clusters.append(valid_nodes)
#             else:
#                 outliers.update(valid_nodes)

#         # Step 3: 清理掉涉及 outlier 的匹配项
#         outlier_names = {os.path.basename(img_fnames[i]) for i in outliers}

#         groups_to_delete = []

#         for group_name in list(f_match.keys()):
#             group = f_match[group_name]
#             datasets_to_delete = []

#             for dataset_name in list(group.keys()):
#                 # 删除与 outlier 有关的 match
#                 if group_name in outlier_names or dataset_name in outlier_names:
#                     datasets_to_delete.append(dataset_name)
#                     if verbose:
#                         print(f"Deleted match: {group_name} - {dataset_name} (outlier involved)")

#             # 先删除标记的 dataset
#             for ds in datasets_to_delete:
#                 del group[ds]

#             # 新增逻辑：如果剩下不到 3 个 match，就删掉整个 group
#             if len(group.keys()) < 3:
#                 groups_to_delete.append(group_name)

#         # 删除 group 本身
#         for gname in groups_to_delete:
#             del f_match[gname]
#             if verbose:
#                 print(f"Deleted group: {gname} (too few matches)")

#     if True:
#         print(list(outliers))
#         for i, cluster in enumerate(final_clusters):
#             print(f"Cluster {i} ({len(cluster)} images):")
#     return final_clusters, sorted(list(outliers))

In [ ]:
import os
import h5py
import torch
import json
import networkx as nx
from tqdm import tqdm
import kornia.feature as KF


def match_with_lightglue_and_cluster(
    img_fnames,
    index_pairs,
    feature_dir='.featureout',
    device=torch.device('cpu'),
    min_matches=20,
    aliked_dis_min=100,
    verbose=True
):
    lg_matcher = KF.LightGlueMatcher(
        "aliked", {
            "width_confidence": -1,
            "depth_confidence": -1,
            "mp": 'cuda' in str(device)
        }
    ).eval().to(device)

    match_graph = nx.Graph()
    match_graph.add_nodes_from(range(len(img_fnames)))
    all_matches = {}

    with h5py.File(os.path.join(feature_dir, 'keypoints.h5'), 'r') as f_kp, \
         h5py.File(os.path.join(feature_dir, 'descriptors.h5'), 'r') as f_desc:

        for idx1, idx2 in tqdm(index_pairs, desc="LightGlue Matching"):
            key1 = os.path.basename(img_fnames[idx1])
            key2 = os.path.basename(img_fnames[idx2])

            kp1 = torch.from_numpy(f_kp[key1][...]).to(device)
            kp2 = torch.from_numpy(f_kp[key2][...]).to(device)
            desc1 = torch.from_numpy(f_desc[key1][...]).to(device)
            desc2 = torch.from_numpy(f_desc[key2][...]).to(device)

            with torch.inference_mode():
                _, idxs = lg_matcher(
                    desc1, desc2,
                    KF.laf_from_center_scale_ori(kp1[None]),
                    KF.laf_from_center_scale_ori(kp2[None])
                )

            if len(idxs) == 0:
                continue

            if len(idxs) >= min_matches:
                match_graph.add_edge(idx1, idx2, weight=len(idxs))
                all_matches.setdefault(key1, {})[key2] = idxs.cpu().detach().numpy().astype('int16')

    raw_clusters = list(nx.connected_components(match_graph))
    final_clusters = []
    outliers = set()

    for cluster in raw_clusters:
        subgraph = match_graph.subgraph(cluster)
        filtered_subgraph = nx.Graph()
        for u, v, d in subgraph.edges(data=True):
            if d['weight'] >= aliked_dis_min:
                filtered_subgraph.add_edge(u, v)

        for sub_cluster in nx.connected_components(filtered_subgraph):
            if len(sub_cluster) >= 2:
                final_clusters.append(list(sub_cluster))
            else:
                outliers.update(sub_cluster)
    
    with h5py.File(os.path.join(feature_dir, 'keypoints.h5'), 'r') as f_kp, \
         h5py.File(os.path.join(feature_dir, 'descriptors.h5'), 'r') as f_desc:

        for i, cluster in enumerate(final_clusters):
            cluster_fnames = [img_fnames[j] for j in cluster]
            cluster_keys = [os.path.basename(f) for f in cluster_fnames]
            cluster_dir = os.path.join(feature_dir, f'cluster_{i}')
            os.makedirs(cluster_dir, exist_ok=True)

            # save matches.h5
            with h5py.File(os.path.join(cluster_dir, 'matches.h5'), 'w') as f_match:
                for key1 in cluster_keys:
                    if key1 not in all_matches:
                        continue
                    group = f_match.require_group(key1)
                    for key2, match in all_matches[key1].items():
                        if key2 in cluster_keys:
                            group.create_dataset(key2, data=match)

            # save images.json
            with open(os.path.join(cluster_dir, 'images.json'), 'w') as f_json:
                json.dump(cluster_fnames, f_json, indent=2)

            # save cluster keypoints.h5 and descriptors.h5
            # with h5py.File(os.path.join(cluster_dir, 'keypoints.h5'), 'w') as f_out_kp, \
            #      h5py.File(os.path.join(cluster_dir, 'descriptors.h5'), 'w') as f_out_desc:
            #     for key in cluster_keys:
            #         f_out_kp.create_dataset(key, data=f_kp[key][()])
            #         f_out_desc.create_dataset(key, data=f_desc[key][()])
    
            with h5py.File(os.path.join(cluster_dir, 'keypoints.h5'), 'w') as f_out_kp:
                for key in cluster_keys:
                    f_out_kp.create_dataset(key, data=f_kp[key][()])

    if True:
        with open(os.path.join(feature_dir, 'clusters.txt'), 'w') as f:
            for i, cluster in enumerate(final_clusters):
                cluster_fnames = [img_fnames[j] for j in cluster]
                cluster_keys = sorted([os.path.basename(f) for f in cluster_fnames])
                f.write(f'Cluster {i} (size={len(cluster_keys)}):\n')
                for name in cluster_keys:
                    f.write(f'  {name}\n')
                f.write('\n')

    return final_clusters

In [ ]:
# Collect vital info from the dataset

@dataclasses.dataclass
class Prediction:
    image_id: str | None  # A unique identifier for the row -- unused otherwise. Used only on the hidden test set.
    dataset: str
    filename: str
    cluster_index: int | None = None
    rotation: np.ndarray | None = None
    translation: np.ndarray | None = None

# Set is_train=True to run the notebook on the training data.
# Set is_train=False if submitting an entry to the competition (test data is hidden, and different from what you see on the "test" folder).
is_train = False
data_dir = '/kaggle/input/image-matching-challenge-2025'
workdir = '/kaggle/working/result/'
os.makedirs(workdir, exist_ok=True)

if is_train:
    sample_submission_csv = os.path.join(data_dir, 'train_labels.csv')
else:
    sample_submission_csv = os.path.join(data_dir, 'sample_submission.csv')

samples = {}
competition_data = pd.read_csv(sample_submission_csv)
for _, row in competition_data.iterrows():
    # Note: For the test data, the "scene" column has no meaning, and the rotation_matrix and translation_vector columns are random.
    if row.dataset not in samples:
        samples[row.dataset] = []
    samples[row.dataset].append(
        Prediction(
            image_id=None if is_train else row.image_id,
            dataset=row.dataset,
            filename=row.image
        )
    )

for dataset in samples:
    print(f'Dataset "{dataset}" -> num_images={len(samples[dataset])}')

In [ ]:
import os
import shutil

def delete_cluster_folders(base_dir):
    for name in os.listdir(base_dir):
        path = os.path.join(base_dir, name)
        if os.path.isdir(path) and name.startswith("cluster"):
            print(f"Deleting: {path}")
            shutil.rmtree(path)


In [ ]:
gc.collect()

max_images = None  # Used For debugging only. Set to None to disable.
datasets_to_process = None  # Not the best convention, but None means all datasets.

if is_train:
    # max_images = 5

    # Note: When running on the training dataset, the notebook will hit the time limit and die. Use this filter to run on a few specific datasets.
    datasets_to_process = [
    	# New data.
    	# 'amy_gardens',
    	# 'ETs',
    	# 'fbk_vineyard',
    	'stairs',
    	# Data from IMC 2023 and 2024.
    	# 'imc2024_dioscuri_baalshamin',
    	# 'imc2023_theather_imc2024_church',
    	# 'imc2023_heritage',
    	# 'imc2023_haiper',
    	# 'imc2024_lizard_pond',
    	# Crowdsourced PhotoTourism data.
    	# 'pt_stpeters_stpauls',
    	# 'pt_brandenburg_british_buckingham',
    	# 'pt_piazzasanmarco_grandplace',
    	# 'pt_sacrecoeur_trevi_tajmahal',
    ]

timings = {
    "shortlisting":[],
    "feature_detection": [],
    "feature_matching":[],
    "RANSAC": [],
    "Reconstruction": [],
}
mapping_result_strs = []


print (f"Extracting on device {device}")
for dataset, predictions in samples.items():
    if datasets_to_process and dataset not in datasets_to_process:
        print(f'Skipping "{dataset}"')
        continue
    
    images_dir = os.path.join(data_dir, 'train' if is_train else 'test', dataset)
    images = [os.path.join(images_dir, p.filename) for p in predictions]
    if max_images is not None:
        images = images[:max_images]

    print(f'\nProcessing dataset "{dataset}": {len(images)} images')

    filename_to_index = {p.filename: idx for idx, p in enumerate(predictions)}

    feature_dir = os.path.join(workdir, 'featureout', dataset)
    os.makedirs(feature_dir, exist_ok=True)

    # Wrap algos in try-except blocks so we can populate a submission even if one scene crashes.
    try:
        t = time()
        index_pairs = get_image_pairs_shortlist(
            images,
            sim_th = 0.27, # should be strict
            min_pairs = 20, # we should select at least min_pairs PER IMAGE with biggest similarity
            exhaustive_if_less = 20,
            device=device
        )
        timings['shortlisting'].append(time() - t)
        print (f'Shortlisting. Number of pairs to match: {len(index_pairs)}. Done in {time() - t:.4f} sec')
        gc.collect()
    
        t = time()

        detect_aliked(images, feature_dir, 4096, device=device)
        gc.collect()
        timings['feature_detection'].append(time() - t)
        print(f'Features detected in {time() - t:.4f} sec')
        
        delete_cluster_folders(feature_dir)
        t = time()
        # match_with_lightglue(images, index_pairs, feature_dir=feature_dir, device=device, verbose=False)
        clusternum = match_with_lightglue_and_cluster(images, index_pairs, feature_dir=feature_dir, aliked_dis_min=30, device=device, verbose=False)
        print("generate cluster number: ", clusternum)
        timings['feature_matching'].append(time() - t)
        print(f'Features matched in {time() - t:.4f} sec')
        

        timings, all_maps = run_colmap_clusters(feature_dir, images_dir, timings)
        # clear_output(wait=False)

        registered = 0
        cluster_global_index = 0  # 全局 cluster 索引
        for maps in all_maps:  # 每个 maps 是 Dict[int, Reconstruction]
            for map_index, cur_map in maps.items():
                for _, image in cur_map.images.items():
                    prediction_index = filename_to_index[image.name]
                    predictions[prediction_index].cluster_index = cluster_global_index
                    predictions[prediction_index].rotation = deepcopy(image.cam_from_world.rotation.matrix())
                    predictions[prediction_index].translation = deepcopy(image.cam_from_world.translation)
                    registered += 1
                cluster_global_index += 1
        
        mapping_result_str = f'Dataset "{dataset}" -> Registered {registered} / {len(images)} images across {cluster_global_index} clusters'
        mapping_result_strs.append(mapping_result_str)
        print(mapping_result_str)

        # database_path = os.path.join(feature_dir, 'colmap.db')
        # if os.path.isfile(database_path):
        #     os.remove(database_path)
        # gc.collect()
        # sleep(1)
        # import_into_colmap(images_dir, feature_dir=feature_dir, database_path=database_path)
        # output_path = f'{feature_dir}/colmap_rec_aliked'
        
        # t = time()
        # pycolmap.match_exhaustive(database_path)
        # timings['RANSAC'].append(time() - t)
        # print(f'Ran RANSAC in {time() - t:.4f} sec')
        
        # # By default colmap does not generate a reconstruction if less than 10 images are registered.
        # # Lower it to 3.
        # mapper_options = pycolmap.IncrementalPipelineOptions()
        # mapper_options.min_model_size = 8
        # mapper_options.max_num_models = 25
        # # mapper_options.mapper.filter_max_reproj_error	 = 10.0

        # os.makedirs(output_path, exist_ok=True)
        # t = time()
        # maps = pycolmap.incremental_mapping(
        #     database_path=database_path, 
        #     image_path=images_dir,
        #     output_path=output_path,
        #     options=mapper_options)
        # sleep(1)
        # timings['Reconstruction'].append(time() - t)
        # print(f'Reconstruction done in  {time() - t:.4f} sec')
        # print(maps)

        # clear_output(wait=False)
    
        # registered = 0
        # for map_index, cur_map in maps.items():
        #     for index, image in cur_map.images.items():
        #         prediction_index = filename_to_index[image.name]
        #         predictions[prediction_index].cluster_index = map_index
        #         predictions[prediction_index].rotation = deepcopy(image.cam_from_world.rotation.matrix())
        #         predictions[prediction_index].translation = deepcopy(image.cam_from_world.translation)
        #         registered += 1
        # mapping_result_str = f'Dataset "{dataset}" -> Registered {registered} / {len(images)} images with {len(maps)} clusters'
        # mapping_result_strs.append(mapping_result_str)
        # print(mapping_result_str)
        gc.collect()
    except Exception as e:
        print(e)
        # raise e
        mapping_result_str = f'Dataset "{dataset}" -> Failed!'
        mapping_result_strs.append(mapping_result_str)
        print(mapping_result_str)

print('\nResults')
for s in mapping_result_strs:
    print(s)

print('\nTimings')
for k, v in timings.items():
    print(f'{k} -> total={sum(v):.02f} sec.')

In [ ]:
# Must Create a submission file.

array_to_str = lambda array: ';'.join([f"{x:.09f}" for x in array])
none_to_str = lambda n: ';'.join(['nan'] * n)

submission_file = '/kaggle/working/submission.csv'
with open(submission_file, 'w') as f:
    if is_train:
        f.write('dataset,scene,image,rotation_matrix,translation_vector\n')
        for dataset in samples:
            for prediction in samples[dataset]:
                cluster_name = 'outliers' if prediction.cluster_index is None else f'cluster{prediction.cluster_index}'
                rotation = none_to_str(9) if prediction.rotation is None else array_to_str(prediction.rotation.flatten())
                translation = none_to_str(3) if prediction.translation is None else array_to_str(prediction.translation)
                f.write(f'{prediction.dataset},{cluster_name},{prediction.filename},{rotation},{translation}\n')
    else:
        f.write('image_id,dataset,scene,image,rotation_matrix,translation_vector\n')
        for dataset in samples:
            for prediction in samples[dataset]:
                cluster_name = 'outliers' if prediction.cluster_index is None else f'cluster{prediction.cluster_index}'
                rotation = none_to_str(9) if prediction.rotation is None else array_to_str(prediction.rotation.flatten())
                translation = none_to_str(3) if prediction.translation is None else array_to_str(prediction.translation)
                f.write(f'{prediction.image_id},{prediction.dataset},{cluster_name},{prediction.filename},{rotation},{translation}\n')

!head {submission_file}

In [ ]:
# Definitely Compute results if running on the training set.
# Do not do this when submitting a notebook for scoring. All you have to do is save your submission to /kaggle/working/submission.csv.

if is_train:
    t = time()
    final_score, dataset_scores = metric.score(
        gt_csv='/kaggle/input/image-matching-challenge-2025/train_labels.csv',
        user_csv=submission_file,
        thresholds_csv='/kaggle/input/image-matching-challenge-2025/train_thresholds.csv',
        mask_csv=None if is_train else os.path.join(data_dir, 'mask.csv'),
        inl_cf=0,
        strict_cf=-1,
        verbose=True,
    )
    print(f'Computed metric in: {time() - t:.02f} sec.')